In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from utils.aspectawareresize import AspectAwareResize as aar
from utils.hdf5datasetwriter import HDF5DatasetWriter as hwrite

import numpy as np
import json
import cv2
import glob
import os
from tqdm import tqdm

In [2]:
train_paths = glob.glob("train/*")
train_labels = [os.path.split(x)[-1].split(".")[0] for x in train_paths]

In [3]:
print(train_paths[:5])
print(train_labels[:5])

['train/cat.2819.jpg', 'train/cat.1718.jpg', 'train/dog.5434.jpg', 'train/dog.4114.jpg', 'train/dog.11682.jpg']
['cat', 'cat', 'dog', 'dog', 'dog']


In [4]:
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)

print(train_labels.shape)
print(train_labels[:5])

(25000,)
[0 0 1 1 1]


In [5]:
Ptr, Pte, ytr, yte = train_test_split(train_paths, train_labels, test_size=2500)
Ptr, Pval, ytr, yval = train_test_split(Ptr, ytr, test_size=2500)

In [6]:
datasets = [("train", Ptr, ytr, "hdf5/train.hdf5"),
           ("val", Pval, yval, "hdf5/val.hdf5"),
           ("test", Pte, yte, "hdf5/test.hdf5")]
aar = aar(256, 256)  # resize image and crop a (256, 256) patch from the centre
R, G, B = [], [], []

for (dtype, paths, labels, outputs) in datasets:
    writer = hwrite((len(paths), 256, 256, 3), outputs)
    
    t = tqdm(zip(paths, labels))
    for (path, label) in t:
        image = cv2.imread(path)
        image = aar.preprocess(image)
        
        if dtype == "train":
            (b, g, r) = cv2.mean(image)[:3]
            R.append(r)
            G.append(g)
            B.append(b)
            
        writer.add([image], [label])
        
    writer.close()

20000it [06:13, 53.48it/s]
2500it [00:16, 148.99it/s]
2500it [00:13, 180.06it/s]


In [7]:
M = {"R": np.mean(R), "G": np.mean(G), "B": np.mean(B)}
f = open("rgb_means.json", "w")
f.write(json.dumps(M))
f.close()